# 第4回講義 宿題

## 課題
今Lessonで学んだことを元に, MNISTのファッション版 (Fashion MNIST, クラス数10) を多層パーセプトロン (MLP) によって分類してみましょう.

Fashion MNISTの詳細については以下のリンクを参考にしてください.

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値
Accuracy: 85%

### ルール
- **下のセルで指定されている`x_train、y_train`以外の学習データは使わないでください。**
- **MLPのアルゴリズム部分の実装はnumpyのみで行ってください** (sklearnやtensorflowなどは使用しないでください)。
    - データの前処理部分でsklearnの関数を使う (例えば `sklearn.model_selection.train_test_split`) のは問題ありません。

### 提出方法
- 2つのファイルを提出していただきます。
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し、**Homeworkタブから`chap04`を選択して**提出してください。
    2. それに対応するpythonのコードを`submission_code.py`として保存し、**Homeworkタブから`chap04 (code)`を選択して**提出してください。
      - セルに書いたコードを.py形式で保存するためには%%writefileコマンドなどを利用してください（writefileコマンドではファイルの保存のみが行われセル内のpythonコード自体は実行されません。そのため、実際にコードを走らせる際にはwritefileコマンドをコメントアウトしてください）。
      
- なお、採点は1で行い、2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）。コードの内容を変更した場合は、**1と2の両方を提出し直してください**。

### 評価方法
- 予測ラベルの`y_test`に対する精度 (Accuracy) で評価します.
- 毎日夜24時にテストデータの一部に対する精度でLeader Boardを更新します.
- 締切日の夜24時にテストデータ全体に対する精度でLeader Boardを更新します. これを最終的な評価とします.

### データの読み込み
- この部分は修正しないでください。

In [2]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

def load_fashionmnist():
    # 学習データ
    x_train = np.load('/root/userspace/public/chap04/data/x_train.npy')
    y_train = np.load('/root/userspace/public/chap04/data/y_train.npy')
    
    # テストデータ
    x_test = np.load('/root/userspace/public/chap04/data/x_test.npy')

    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255
    
    return x_train, y_train, x_test

### 多層パーセプトロンの実装

In [22]:
# %%writefile /root/userspace/chap04/materials/submission_code.py

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

x_train, y_train, x_test = load_fashionmnist()

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=1000)

def np_log(x):
    return np.log(np.clip(a = x,a_min =  1e-10, a_max = x))

def relu(x):
    return np.maximum(x, 0)

def deriv_relu(x):
    return (x > 0).astype(x.dtype)

def softmax(x):
    x -= x.max(axis=1, keepdims=True) 
    x_exp = np.exp(x)
    return x_exp / np.sum(x_exp, axis=1, keepdims=True)

def deriv_softmax(x):
    return softmax(x) * (1 - softmax(x))

class Dense:
    # コンストラクタ
    def __init__(self, in_dim, out_dim, function, deriv_func):
        self.W = np.random.uniform(low = -0.08, high = 0.08,
                                  size = (in_dim, out_dim)).astype('float64')
        self.b = np.zeros((out_dim,),dtype = 'float64')
        self.function = function
        self.deriv_func = deriv_func
        
        self.x = None
        self.u = None
        
        self.dW = None
        self.db = None
        
        self.params_idxs = np.cumsum([self.W.size, self.b.size]) #(Wの要素数, Wの要素数+bの要素数)を返す
    
    # 順伝播
    def __call__(self, x):
        self.x = x
        self.u = np.matmul(self.x, self.W) + self.b
        return self.function(self.u)
    
    # 誤差逆伝播
    def b_prop(self, delta, W): ## (l+1)層目のdeltaとWを引数として受け取る
        # print("self.u's shape is {:}, delta's shape is {:}, W's shape is {:}".format(self.u.shape, delta.shape, W.shape))
        self.delta = self.deriv_func(self.u) * np.matmul(delta, W.T)
        return self.delta
    
    # deltaが決定後、勾配計算
    def compute_grad(self):
        batch_size = self.delta.shape[0]
        # print("batch_size is given by {:}".format(batch_size))
        self.dW = np.matmul(self.x.T, self.delta)/batch_size
        ## ここの次元の合わせ方に注意
        self.db = np.matmul(self.delta.T, np.ones((batch_size,)))/batch_size
    
    # パラメータ更新 
    # np.concatenateは２つ以上の配列の結合を行う。axisの方向にくっつける。
    # 対象が２次元配列の場合、axis = 0なら縦に, axis = 1なら横にくっつく
    # numpy.ndarray.ravel(order = "C" or "F" or...)
    # 効果はflattenと同じだが、コピーを取らない。
    # Cなら列方向から読み取り、Fなら行方向から読み取り一次元配列を返す
    def get_params(self):
        return np.concanate([self.W.ravel(), self.b], axis = 0)
    
    # パラメータ設定
    # np.split(params, idxs)はparamsで与えられた一次元配列をidxsで与えられた番号で切って
    # 配列のリストとして返す。
    # idxsの最後の要素はparamsの全長と一致してしまい、np.splitで返される最後の配列は空になるので
    # [:-1]で最後の要素を除外している。
    # 従って、idxs[:-1]と指定しても同じ結果が得られる。
    def set_params(self, params):
        _W, _b = np.split(params, self.params_idxs)[:-1]
        self.W = _W.reshape(self.W.shape)
        self.b = _b
    
    def get_grads(self):
        return np.concatenate([self.dW.ravel(), self.db], axis = 0)
    
    
# 順伝播
# layer(x)は指定されたDense classで__call__()を呼び出している
def f_props(layers, x):
    for layer in layers:
        x = layer(x)
    return x

# 逆伝播
def b_props(layers, delta):
    batch_size = delta.shape[0]
    for i, layer in enumerate(layers[::-1]):
        if i == 0: #出力層
            layer.delta = delta #y-t
            layer.compute_grad()
        else: #隠れ層
            delta = layer.b_prop(delta, W)
            layer.compute_grad()
        
        W = layer.W
        

def update_parameters(layers, eps):
    for layer in layers:
        layer.W -= eps * layer.dW
        layer.b -= eps * layer.db

# 取り敢えず、3層で計算してみる
layers = [
    Dense(784, 100, relu, deriv_relu),
    Dense(100, 100, relu, deriv_relu),
    Dense(100, 10, softmax, deriv_softmax)
]

def train_mnist(x, t, eps):
    y = f_props(layers, x)
    cost = (-t * np_log(y)).sum(axis = 1).mean()
    
    delta = y - t
    b_props(layers, delta)
    update_parameters(layers, eps)
    
    return cost


def valid_mnist(x, t):
    y = f_props(layers, x)
    cost = (- t * np_log(y)).sum(axis = 1).mean()
    return cost, y

batch_size = 80
num_epoch = 100
num_data = x_train.shape[0]
for epoch in range(num_epoch):
    # minibatch learning
    x_train_fmnist, y_train_fmnist = shuffle(x_train, y_train)
    for idx in range(0, num_data, batch_size):
        cost = train_mnist(x_train_fmnist[idx: min(idx + batch_size, num_data), :] \
                , y_train_fmnist[idx: min(idx + batch_size, num_data), :],
                          eps = 0.01)
    cost, y_valid_pred = valid_mnist(x_valid, y_valid)
    accuracy = accuracy_score(y_valid.argmax(axis = 1),y_valid_pred.argmax(axis = 1))
    print('Batch_size: {}, EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
        batch_size,
        epoch + 1,
        cost,
        accuracy
    ))

y_pred = f_props(layers, x_test)
y_pred = np.argmax(y_pred, axis = 1)

submission = pd.Series(y_pred, name='label')
submission.to_csv('/root/userspace/chap04/submission/submission_pred_minibatch.csv', header=True, index_label='id')

Writing /root/userspace/chap04/materials/submission_code.py


### オンライン学習
1回目の結果は(784,100,100,10)のNNで  
EPOCH: 1, Valid Cost: 0.502, Valid Accuracy: 0.825  
EPOCH: 2, Valid Cost: 0.419, Valid Accuracy: 0.857  
EPOCH: 3, Valid Cost: 0.390, Valid Accuracy: 0.863  
EPOCH: 4, Valid Cost: 0.395, Valid Accuracy: 0.865  
EPOCH: 5, Valid Cost: 0.401, Valid Accuracy: 0.870  
でした

In [ ]:

for epoch in range(5):
    x_train, y_train = shuffle(x_train, y_train)
    # オンライン学習
    for x, t in zip(x_train, y_train):
        cost = train_mnist(x[None,:], t[None, :], eps = 0.01)
    
    # cv
    cost, y_pred = valid_mnist(x_valid, y_valid)
    accuracy = accuracy_score(y_valid.argmax(axis = 1), y_pred.argmax(axis = 1))
    print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(epoch + 1, cost, accuracy))

y_pred = f_props(layers, x_test)
y_pred = np.argmax(y_pred, axis=1)

submission = pd.Series(y_pred, name='label')
submission.to_csv('/root/userspace/chap04/submission/submission_pred.csv', header=True, index_label='id')